Imports necessários:

In [1]:
import pandas as pd

Lendo o arquivo e armazenando em uma lista:

In [2]:
filepath = 'C:\\Users\\User\\Desktop\\8º período\\DM\\TP1\\'

txt_file = open(filepath + 'data\\original\\gen8ou.txt', 'r', encoding="utf8")
content_as_list = txt_file.readlines()
txt_file.close()

Pegando as posições dos delimitadores de equipes no arquivo (cabeçalhos que começam com '==='):

In [3]:
content_as_dict = dict(enumerate(content_as_list))

In [4]:
delimiter_positions = []

for key, value in content_as_dict.items():
    if value.startswith('==='):
        delimiter_positions.append(key)

Criando um dicionário em que a chave é a posição de um delimitador e o valor é a posição do próximo delimitador:

In [5]:
shifted_delimiter_positions = delimiter_positions[1:]
shifted_delimiter_positions.append(277010)
delimiter_positions_dic = dict(zip(delimiter_positions, shifted_delimiter_positions))

Dividindo a lista original em uma lista de blocos de texto que representam as equipes:

In [6]:
team_list = []

for begin, end in delimiter_positions_dic.items():
    team_list.append(content_as_list[begin:end])

Transformando a lista de equipes em uma lista de dicionários em que cada linha do bloco se torna um valor, a fim de facilitar a manipulação posteriormente:

In [7]:
team_as_list_of_dict = []

for i in team_list:
    team_as_list_of_dict.append(dict(enumerate(i)))

Gerando uma lista em que cada elemento é uma lista contendo os nomes dos pokémons da equipe, deixando de lado as informações que não serão utilizadas:

In [8]:
formatted_team_list = []

for block in team_as_list_of_dict:
    team = []
    for index, line in block.items():
        if line.startswith('Ability'): # A linha acima da que começa com 'Ability' é sempre a que contém o nome do pokémon
            team.append(block[index-1].split(' @')[0]) # Deixando de lado as informações sobre item utilizado
    formatted_team_list.append(team)

Pegando apenas equipes de 6 pokémons:

In [9]:
teams_with_six_members = []

for team in formatted_team_list:
    if len(team) == 6:
        teams_with_six_members.append(team)
        
print(f'Existem {len(teams_with_six_members)} equipes de 6 pokémons na base!' )

Existem 4342 equipes de 6 pokémons na base!


Agora vamos remover as informações de sexo dos pokémons, pois essa diferenciação irá apenas nos atrapalhar:

In [10]:
teams_without_male_sex_information = []

for team in teams_with_six_members:
    pokemons = []
    for pokemon in team:
        pokemons.append(pokemon.split(' (M)'))
    teams_without_male_sex_information.append(pokemons)

teams_without_sex_information = []

for team in teams_without_male_sex_information:
    pokemons = []
    for pokemon_as_list in team:
        for element in pokemon_as_list:
            pokemons.append(element.split(' (F)'))
    teams_without_sex_information.append(pokemons)

pokemon_teams = []

for team in teams_without_sex_information:
    pokemons = []
    for pokemon_as_list in team:
        for element in pokemon_as_list:
            if element != [''] and element != '' and element != '  \n' and element != ' \n':
                pokemons.append(element)
    pokemon_teams.append(pokemons)

Separando em dois os nomes dos pokémons que possuem apelidos atribuídos:

In [11]:
teams_with_separate_nicknames = []

for team in pokemon_teams:
    pokemons = []
    for pokemon in team:
        pokemons.append(pokemon.split(' ('))
    teams_with_separate_nicknames.append(pokemons)

Montando um dataframe com as equipes:

In [25]:
columns = ['pokemon_1', 'pokemon_2', 'pokemon_3', 'pokemon_4', 'pokemon_5', 'pokemon_6']
team_df = pd.DataFrame(columns=columns)

for index, team in enumerate(teams_with_separate_nicknames):
    team_df.loc[index] = team
    
team_df.head(15)

,pokemon_1,pokemon_2,pokemon_3,pokemon_4,pokemon_5,pokemon_6
0,[Landorus-Therian],[Corviknight],[Tapu Fini],[Slowking-Galar],[Volcarona],[Dragapult]
1,[Hawlucha],[Ninetales-Alola],[Garchomp],[Cloyster],[Tapu Lele],[Heatran]
2,[Kartana],[Zapdos],[Heatran],[Landorus-Therian],[Dragapult],[Toxapex]
3,[Heatran],[Melmetal],[Tornadus-Therian],[Landorus-Therian],[Slowbro],[Weavile]
4,"[Lumberjack, Diggersby)]","[Wyatt, Shuckle)]","[Leviathan, Dragonite)]","[Shellshock, Blastoise)]","[Quintessa, Tapu Koko)]","[Achilles, Aegislash)]"
5,[Tapu Koko],[Landorus-Therian],[Volcanion],[Kartana],[Weavile],[Buzzwole]
6,[Gyarados],[Heatran],[Garchomp],[Mew],[Zapdos],[Weavile]
7,[Tapu Koko],[Hippowdon],[Tornadus-Therian],[Skarmory],[Reuniclus],[Toxapex]
8,[Silvally-Electric],[Zeraora],[Landorus-Therian],[Tapu Fini],[Tornadus-Therian],[Heatran]
9,[Landorus-Therian],[Corviknight],[Toxapex],[Blacephalon],[Heatran],[Dragapult]


Removendo os apelidos:

In [26]:
columns = team_df.columns.to_list()

for column in columns:
    team_df[column] = [pokemon[1][:-1] if len(pokemon) == 2 else str(pokemon)[2:-2] for pokemon in team_df[column]]
    
team_df.head(15)

,pokemon_1,pokemon_2,pokemon_3,pokemon_4,pokemon_5,pokemon_6
0,Landorus-Therian,Corviknight,Tapu Fini,Slowking-Galar,Volcarona,Dragapult
1,Hawlucha,Ninetales-Alola,Garchomp,Cloyster,Tapu Lele,Heatran
2,Kartana,Zapdos,Heatran,Landorus-Therian,Dragapult,Toxapex
3,Heatran,Melmetal,Tornadus-Therian,Landorus-Therian,Slowbro,Weavile
4,Diggersby,Shuckle,Dragonite,Blastoise,Tapu Koko,Aegislash
5,Tapu Koko,Landorus-Therian,Volcanion,Kartana,Weavile,Buzzwole
6,Gyarados,Heatran,Garchomp,Mew,Zapdos,Weavile
7,Tapu Koko,Hippowdon,Tornadus-Therian,Skarmory,Reuniclus,Toxapex
8,Silvally-Electric,Zeraora,Landorus-Therian,Tapu Fini,Tornadus-Therian,Heatran
9,Landorus-Therian,Corviknight,Toxapex,Blacephalon,Heatran,Dragapult


Limpando nomes que permaneceram com ' \n' no final:

In [27]:
for column in columns:
    team_df[column] = [pokemon[:-3] if pokemon.endswith('  \\n') else pokemon for pokemon in team_df[column]]
    
team_df.head(15)

,pokemon_1,pokemon_2,pokemon_3,pokemon_4,pokemon_5,pokemon_6
0,Landorus-Therian,Corviknight,Tapu Fini,Slowking-Galar,Volcarona,Dragapult
1,Hawlucha,Ninetales-Alola,Garchomp,Cloyster,Tapu Lele,Heatran
2,Kartana,Zapdos,Heatran,Landorus-Therian,Dragapult,Toxapex
3,Heatran,Melmetal,Tornadus-Therian,Landorus-Therian,Slowbro,Weavile
4,Diggersby,Shuckle,Dragonite,Blastoise,Tapu Koko,Aegislash
5,Tapu Koko,Landorus-Therian,Volcanion,Kartana,Weavile,Buzzwole
6,Gyarados,Heatran,Garchomp,Mew,Zapdos,Weavile
7,Tapu Koko,Hippowdon,Tornadus-Therian,Skarmory,Reuniclus,Toxapex
8,Silvally-Electric,Zeraora,Landorus-Therian,Tapu Fini,Tornadus-Therian,Heatran
9,Landorus-Therian,Corviknight,Toxapex,Blacephalon,Heatran,Dragapult


Finalmente, base de dados tratada \o/

Vamos exportá-la como .csv e .xlsx:

In [28]:
team_df.to_csv(filepath + 'data\\processed\\pokemon_teams.csv', index=False)
team_df.to_excel(filepath + 'data\\processed\\pokemon_teams.xlsx', index=False)